This is my attempt to improve my first attempt at creating an impleid volatility surface with extensive guidance from AI on where my code has gone wrong. Results are still quite poor. Will need to extensively revisit my methodology and code to get a clear volatility surface.

In [ ]:
# ==========================================
# Implied Volatility Surface (Yahoo Finance)
# Smooth + robust version (Spyder-friendly)
# ==========================================

import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.interpolate import griddata, Rbf
import matplotlib.pyplot as plt
import plotly.graph_objects as go

pd.options.mode.chained_assignment = None  # hush copy warnings


# -----------------------------
# Black–Scholes (call) + IV
# -----------------------------
def bs_call_option(S, K, T, r, sigma, q=0.0):
    # assumes T>0, sigma>0 when called from the root finder
    d1 = (np.log(S/K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)


def implied_vol(price, S, K, T, r, q=0.0):
    """
    Robust IV solver with no-arb checks and bracket validation.
    Returns np.nan if no real solution.
    """
    if (T <= 0) or (price <= 0) or (S <= 0) or (K <= 0):
        return np.nan

    # No-arbitrage bounds for a call
    intrinsic = max(S*np.exp(-q*T) - K*np.exp(-r*T), 0.0)
    upper     = S*np.exp(-q*T)
    if (price < intrinsic - 1e-6) or (price > upper + 1e-6):
        return np.nan

    def f(sig):
        return bs_call_option(S, K, T, r, sig, q) - price

    lo, hi = 1e-6, 5.0
    f_lo, f_hi = f(lo), f(hi)
    if np.isnan(f_lo) or np.isnan(f_hi) or f_lo * f_hi > 0:
        return np.nan

    try:
        return brentq(f, lo, hi, maxiter=200, xtol=1e-8)
    except Exception:
        return np.nan


# -----------------------------
# Time, spot, r, q helpers
# -----------------------------
def act365(now_ts, expiry_ts):
    """
    ACT/365 using *days* difference as requested, both UTC-aware.
    """
    now = pd.to_datetime(now_ts, utc=True)
    exp = pd.to_datetime(expiry_ts, utc=True)
    return (exp - now).days / 365.0


def get_spot(ticker):
    tk = yf.Ticker(ticker)
    # Try fast_info
    try:
        px = tk.fast_info.get("last_price", None)
        if px and px > 0:
            return float(px)
    except Exception:
        pass
    # Fallback to recent intraday close
    h = tk.history(period="1d", interval="1m")
    if not h.empty:
        return float(h["Close"].dropna().iloc[-1])
    # Fallback to info
    info = tk.info
    return float(info.get("regularMarketPrice"))


def get_risk_free():
    """
    ^IRX (13w T-bill) as a crude proxy; convert to continuous compounding.
    """
    try:
        irx = yf.Ticker("^IRX").history(period="5d")["Close"].dropna().iloc[-1] / 100.0
        r = np.log(1.0 + float(irx))
        return float(np.clip(r, -0.02, 0.10))
    except Exception:
        return 0.02


def get_dividend_yield(ticker):
    try:
        info = yf.Ticker(ticker).info
        dy = info.get("dividendYield")
        return float(dy) if dy is not None else 0.0
    except Exception:
        return 0.0


# -----------------------------
# Yahoo option chain download
# -----------------------------
def download_option_chain(ticker):
    tk = yf.Ticker(ticker)
    expiries = tk.options or []
    frames = []
    for ex in expiries:
        oc = tk.option_chain(ex)
        for typ, df in [("C", oc.calls), ("P", oc.puts)]:
            if df is None or df.empty:
                continue
            d = df.copy()
            d["type"] = typ
            d["expiration"] = pd.to_datetime(ex, utc=True)
            frames.append(d)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()


# -----------------------------
# Clean, compute T, infer q, synthetic calls
# -----------------------------
def clean_and_prep(raw, ticker, infer_q=True):
    if raw.empty:
        return raw

    now = pd.Timestamp.now(tz="UTC").normalize()
    spot = get_spot(ticker)
    r = get_risk_free()
    q_guess = get_dividend_yield(ticker)

    df = raw.copy()
    # Ensure UTC-aware expiration
    df["expiration"] = pd.to_datetime(df["expiration"], utc=True)

    # Mid & price
    has_ba = df["bid"].notna() & df["ask"].notna() & (df["bid"] >= 0) & (df["ask"] >= 0)
    df["mid"] = np.where(has_ba, 0.5*(df["bid"] + df["ask"]), np.nan)
    df["price"] = df["mid"].where(df["mid"].notna(), df["lastPrice"])

    # T = days/365 (ACT/365)
    df["T"] = df["expiration"].apply(lambda ex: act365(now, ex))

    # Base filters: penny options, tiny T, invalid prices/strikes
    df = df[df["strike"] > 0]
    df = df[df["price"] >= 0.02]
    df = df[df["T"] >= 2/365]  # drop options expiring within ~2 days
    # Spread filter
    ok_spread = (~has_ba) | ((df["ask"] - df["bid"]) <= 0.6*df["price"])
    df = df[ok_spread]
    # Liquidity hint
    liq = (df["openInterest"].fillna(0) > 0) | (df["volume"].fillna(0) > 0) | has_ba
    df = df[liq]

    # Attach S, r, q (initial)
    df["spot"] = spot
    df["r"] = r
    df["q"] = q_guess

    # Optional: refine q per expiry via near-ATM parity
    if infer_q:
        def parity_refine(group):
            T = float(group["T"].iloc[0])
            if T <= 0:
                return group
            S = float(group["spot"].iloc[0])
            r_ = float(group["r"].iloc[0])
            # pick near-ATM strikes around a forward guess
            F_guess = S * np.exp((r_ - q_guess) * T)
            idx = (group["strike"] - F_guess).abs().sort_values().index[:5]
            g = group.loc[idx]

            calls = g[g["type"] == "C"]["price"]
            puts  = g[g["type"] == "P"]["price"]
            Ks    = g["strike"]
            if calls.empty or puts.empty:
                return group

            K_med = float(Ks.median())
            C_med = float(group[(group["type"]=="C") & (group["strike"]==K_med)]["price"].median())
            P_med = float(group[(group["type"]=="P") & (group["strike"]==K_med)]["price"].median())
            if not np.isfinite(C_med) or not np.isfinite(P_med):
                return group

            try:
                q_hat = r_ - (1.0/T)*np.log((C_med - P_med + K_med*np.exp(-r_*T))/S)
                if np.isfinite(q_hat) and -0.05 <= q_hat <= 0.20:
                    group["q"] = q_hat
            except Exception:
                pass
            return group

        df = df.groupby("expiration", group_keys=False).apply(parity_refine)

    # Synthetic call price for puts: C = P + S e^{-qT} - K e^{-rT}
    df["price_call_equiv"] = np.where(
        df["type"] == "C",
        df["price"],
        df["price"] + df["spot"]*np.exp(-df["q"]*df["T"]) - df["strike"]*np.exp(-df["r"]*df["T"])
    )

    return df[[
        "expiration","T","type","strike","bid","ask","price","price_call_equiv",
        "openInterest","volume","impliedVolatility","spot","r","q"
    ]].reset_index(drop=True)


# -----------------------------
# IV calculation + moneyness
# -----------------------------
def add_implied_vols(df):
    df = df.copy()
    ivs = []
    for _, row in df.iterrows():
        S = float(row["spot"]); K = float(row["strike"]); T = float(row["T"])
        r = float(row["r"]);     q = float(row["q"])
        C = float(row["price_call_equiv"])

        intrinsic = max(S*np.exp(-q*T) - K*np.exp(-r*T), 0.0)
        upper     = S*np.exp(-q*T)
        if (C <= intrinsic + 1e-6) or (C >= upper - 1e-6):
            iv = np.nan
        else:
            iv = implied_vol(C, S, K, T, r, q)
        ivs.append(iv)

    df["iv"] = ivs
    df = df[(df["iv"] > 1e-4) & (df["iv"] < 3.0)]

    # Forward & log-moneyness k = ln(K/F)
    F = df["spot"] * np.exp((df["r"] - df["q"]) * df["T"])
    df["k"] = np.log(df["strike"] / F)
    return df


# -----------------------------
# Surface grid + smoothing
# -----------------------------
def make_surface_grid(df, n_T=40, n_k=60, smoothing_alpha=0.30, rbf_smooth=0.002, rbf_eps=0.15):
    """
    Build a (T,k) grid:
    1) linear griddata (good within convex hull)
    2) fill holes with nearest
    3) optional RBF smoothing blended with weight alpha
    """
    T_vals = np.linspace(df["T"].min(), df["T"].max(), n_T)
    k_vals = np.linspace(df["k"].quantile(0.05), df["k"].quantile(0.95), n_k)
    TT, KK = np.meshgrid(T_vals, k_vals, indexing="xy")

    pts = np.column_stack((df["T"].values, df["k"].values))
    IV_lin = griddata(pts, df["iv"].values, (TT, KK), method="linear")
    IV_near = griddata(pts, df["iv"].values, (TT, KK), method="nearest")
    IV_grid = np.where(np.isnan(IV_lin), IV_near, IV_lin)

    if smoothing_alpha and smoothing_alpha > 0:
        rbf = Rbf(df["T"].values, df["k"].values, df["iv"].values,
                  function="multiquadric", smooth=rbf_smooth, epsilon=rbf_eps)
        IV_rbf = rbf(TT, KK)
        IV_blend = (1 - smoothing_alpha) * IV_grid + smoothing_alpha * IV_rbf
        return TT, KK, IV_blend
    else:
        return TT, KK, IV_grid


# -----------------------------
# Plotting
# -----------------------------
def plot_surface_matplotlib(TT, KK, IV):
    fig = plt.figure(figsize=(9, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(TT, KK, IV, rstride=1, cstride=1, linewidth=0, antialiased=True)
    ax.set_xlabel("T (years)")
    ax.set_ylabel("log-moneyness k = ln(K/F)")
    ax.set_zlabel("Implied Volatility")
    plt.title("Implied Volatility Surface")
    plt.show()


def plot_surface_plotly(TT, KK, IV):
    fig = go.Figure(data=[go.Surface(x=TT, y=KK, z=IV)])
    fig.update_layout(
        title="Implied Volatility Surface",
        scene=dict(
            xaxis_title="T (years)",
            yaxis_title="log-moneyness (k)",
            zaxis_title="Implied Volatility",
        ),
        height=700
    )
    fig.show()


# -----------------------------
# End-to-end run
# -----------------------------
if __name__ == "__main__":
    ticker = "SPY"       # change me
    infer_q = True       # per-expiry dividend yield via parity

    raw = download_option_chain(ticker)
    if raw.empty:
        raise RuntimeError("No option data returned. Try another ticker or rerun.")

    chain = clean_and_prep(raw, ticker, infer_q=infer_q)
    iv_df = add_implied_vols(chain)

    print(f"{ticker}: rows after cleaning & IV solve = {len(iv_df)}")
    print(iv_df.head(10)[["expiration","type","strike","T","price","price_call_equiv","iv","k"]])

    TT, KK, IV = make_surface_grid(
        iv_df,
        n_T=40, n_k=60,
        smoothing_alpha=0.30,   # 0 → no smoothing, 0.3–0.5 is a good start
        rbf_smooth=0.002,
        rbf_eps=0.15
    )

    # Choose either matplotlib or plotly
    plot_surface_matplotlib(TT, KK, IV)
    # plot_surface_plotly(TT, KK, IV)
